In [1]:
import Constants
from  openai import OpenAI
import os
import numpy as np
import pandas as pd
import time

import gspread
import gspread_dataframe as gd
import gspread_formatting as gf
from gspread_formatting import cellFormat, color, textFormat


## Generating Word Quiz

In [2]:
from main.quiz import *

temp = 0.7
dict_sheet_name = "Tua_List"
gsheet_name = "New Chinese Words"

In [3]:
word_dict = load_dict(gsheet_mode=True, gsheet_name=gsheet_name, worksheet_name=dict_sheet_name)
word_dict.head(2)

,Word Id,Word,Pinyin,Pinyin Simplified,Type,Word Category,Word Rarity,Meaning,Sentence,Sentence Pinyin,Sentence Meaning,Added Date,Num_Quiz_Attempt,Num_Correct,Num_Wrong,Last_Quiz
1,1,帮助,bang1 zhu4,bang1 zhu4,Noun/Verb,Support,Common,Help/Assistance,我可以帮助你学习中文,Wǒ kěyǐ bāngzhù nǐ xuéxí Zhōngwén.,I can help you study Chinese.,2024-09-21,0,0,0,
2,2,包裹,bao1 guo3,bao1 guo3,Noun,Object,Common,Package,他昨天收到了一个包裹。,Wǒ zuótiān shōudào le bāoguǒ.,I received the package yesterday.,2024-09-21,0,0,0,


In [17]:
quiz_df_show, answer_key = generate_pinyin_and_meaning_quiz(
    word_dict, 
    id_column='Word Id', 
    random_params={
        'num_words': 10, 
        'date_filter': '2024-10-10', 
        #'category_filter': 'Fashion'
        'rarity_filter': 'Common'
        }
    )

quiz_df_no_id = quiz_df_show.drop(columns=['Word Id'])
quiz_df_no_id

,Word,Sentence
0,安静,图书馆里非常安静。
1,别,别忘了锁门。
2,发烧,他昨天晚上发烧了，今天不能上班。
3,旋转,风车在微风中缓缓旋转。
4,犯,他犯了一个错误。
5,旧,这是一件旧衣服。
6,遇见,我在街上遇见了我的老朋友。
7,脚,他的脚很大。
8,幽默,他的幽默让大家都笑了。
9,生病,他生病了，不能来上课。


In [21]:
pinyin_answer = 'an1jing4, bie2, umm, ded, fan4, jiu2, yu4jian4, jiao3, you1mo4, sheng1bing4'.split(', ')
meaning_answer = 'quiet, dont, ded, to spin, to make a mistake, old, to meet, feet, humor, sick'.split(', ')

In [22]:
quiz_result = evaluate_pinyin_and_meaning_quiz(
    answer_key_df = answer_key, 
    quiz_df = quiz_df_show, 
    pinyin_answer = pinyin_answer, 
    meaning_answer = meaning_answer
)

update_quiz_score(
    quiz_result_df = quiz_result, 
    gsheet_name = gsheet_name, 
    wks_name = 'Tua_List'
    )

quiz_result

c:\Users\wongs\Documents\DS Work\repo\Pytorch LLM\GPT4LanguageLearning\main\quiz.py:125: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,Word,Sentence,Pinyin Answer,Pinyin Correct,Pinyin Correction,Meaning,Meaning Correct,Meaning Correction,Right Score,Wrong Score,Last Quiz
Word Id,,,,,,,,,,,
553,安静,图书馆里非常安静。,an1jing4,yes,,quiet,yes,,1,0,2024-11-13
1226,别,别忘了锁门。,bie2,yes,,dont,no,"Incorrect usage; means ""don't"" or ""other.""",0,1,2024-11-13
944,发烧,他昨天晚上发烧了，今天不能上班。,umm,no,fāshāo,ded,no,"Incorrect; means ""to have a fever.""",0,1,2024-11-13
865,旋转,风车在微风中缓缓旋转。,ded,no,xuán zhuǎn,to spin,yes,,0,1,2024-11-13
324,犯,他犯了一个错误。,fan4,yes,,to make a mistake,yes,,1,0,2024-11-13
825,旧,这是一件旧衣服。,jiu2,no,jiù,old,yes,,0,1,2024-11-13
375,遇见,我在街上遇见了我的老朋友。,yu4jian4,yes,,to meet,yes,,1,0,2024-11-13
957,脚,他的脚很大。,jiao3,yes,,feet,no,"Incorrect; means ""foot.""",0,1,2024-11-13
1000,幽默,他的幽默让大家都笑了。,you1mo4,yes,,humor,yes,,1,0,2024-11-13


##### Check Update Condition

In [ ]:
quiz_result_df = quiz_result, 
gsheet_name = gsheet_name, 
wks_name = 'Tua_List'

word_dict = load_dict(gsheet_mode=True, gsheet_name=gsheet_name, worksheet_name=wks_name)

quiz_result_df['Right Score'] = np.where((quiz_result_df['Meaning Correct']=="yes")&(quiz_result_df['Pinyin Correct']=="yes"), 1, 0)
quiz_result_df['Wrong Score'] = np.where(quiz_result_df['Right Score']==0, 1, 0)
quiz_result_df['Last Quiz'] = datetime.now().strftime('%Y-%m-%d')

word_dict_quiz_export = word_dict.merge(quiz_result_df[['Right Score', 'Wrong Score', 'Last Quiz']].reset_index(), on='Word Id', how='left')

num_attempts = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
word_dict_quiz_export['Num_Quiz_Attempt'] = word_dict_quiz_export['Num_Quiz_Attempt'].astype(int) + word_dict_quiz_export['Right Score'].notna()
word_dict_quiz_export['Num_Correct'] = word_dict_quiz_export['Num_Correct'].astype(int) + word_dict_quiz_export['Right Score'].fillna(0)
word_dict_quiz_export['Num_Wrong'] = word_dict_quiz_export['Num_Wrong'].astype(int) + word_dict_quiz_export['Wrong Score'].fillna(0)
word_dict_quiz_export['Last_Quiz'] = word_dict_quiz_export['Last_Quiz'].fillna('2001-01-01')
word_dict_quiz_export['Last_Quiz'] = np.where(word_dict_quiz_export['Last_Quiz'] < word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last_Quiz'])

num_attempts_post_update = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
num_correct_post_update = word_dict_quiz_export['Num_Correct'].fillna(0).astype(int).sum()
num_wrong_post_update = word_dict_quiz_export['Num_Wrong'].fillna(0).astype(int).sum()

attempt_count_condition = (num_attempts + len(quiz_result_df) != num_attempts_post_update)
correct_wrong_total_condition = (num_correct_post_update + num_wrong_post_update != num_attempts_post_update)

if (attempt_count_condition) or (correct_wrong_total_condition):
    raise Exception("Error in updating the quiz results.  Please check the code.")

word_dict_quiz_export = word_dict_quiz_export.drop(columns=['Right Score', 'Wrong Score', 'Last Quiz'])



## Manual Code

In [ ]:
quiz_result_export = quiz_result.copy()
quiz_result_export['Right Score'] = np.where((quiz_result_export['Meaning Correct']=="yes")&(quiz_result_export['Pinyin Correct']=="yes"), 1, 0)
quiz_result_export['Wrong Score'] = np.where(quiz_result_export['Right Score']==0, 1, 0)
quiz_result_export['Last Quiz'] = datetime.now().strftime('%Y-%m-%d')

quiz_result_export[['Right Score', 'Wrong Score', 'Last Quiz']].reset_index()

In [ ]:
word_dict_quiz_export = word_dict.merge(quiz_result_export[['Right Score', 'Wrong Score', 'Last Quiz']].reset_index(), on='Word Id', how='left')

num_attempts = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
word_dict_quiz_export['Num_Quiz_Attempt'] = word_dict_quiz_export['Num_Quiz_Attempt'].astype(int) + word_dict_quiz_export['Right Score'].notna()
word_dict_quiz_export['Num_Correct'] = word_dict_quiz_export['Num_Correct'].astype(int) + word_dict_quiz_export['Right Score'].fillna(0)
word_dict_quiz_export['Num_Wrong'] = word_dict_quiz_export['Num_Wrong'].astype(int) + word_dict_quiz_export['Wrong Score'].fillna(0)
word_dict_quiz_export['Last_Quiz'] = word_dict_quiz_export['Last_Quiz'].fillna('2001-01-01')
word_dict_quiz_export['Last_Quiz'] = np.where(word_dict_quiz_export['Last_Quiz'] < word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last_Quiz'])

num_attempts_post_update = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
num_correct_post_update = word_dict_quiz_export['Num_Correct'].fillna(0).astype(int).sum()
num_wrong_post_update = word_dict_quiz_export['Num_Wrong'].fillna(0).astype(int).sum()

attempt_count_condition = (num_attempts + len(quiz_result_export) != num_attempts_post_update)
correct_wrong_total_condition = (num_correct_post_update + num_wrong_post_update != num_attempts_post_update)

if (attempt_count_condition) or (correct_wrong_total_condition):
    raise Exception("Error in updating the quiz results.  Please check the code.")

word_dict_quiz_export = word_dict_quiz_export.drop(columns=['Right Score', 'Wrong Score'])